# Supervised Fine-Tuning


1️⃣ Chat Templates

Chat templates structure interactions between users and AI models, ensuring consistent and contextually appropriate responses. They include components like system prompts and role-based messages.

2️⃣ Supervised Fine-Tuning

Supervised Fine-Tuning (SFT) is a critical process for adapting pre-trained language models to specific tasks. It involves training the model on a task-specific dataset with labeled examples.

3️⃣ Low Rank Adaptation (LoRA)

Low Rank Adaptation (LoRA) is a technique for fine-tuning language models by adding low-rank matrices to the model’s layers. This allows for efficient fine-tuning while preserving the model’s pre-trained knowledge. One of the key benefits of LoRA is the significant memory savings it offers, making it possible to fine-tune large models on hardware with limited resources.

4️⃣ Evaluation

Evaluation is a crucial step in the fine-tuning process. It allows us to measure the performance of the model on a task-specific dataset.

**Supervised Fine-Tuning (SFT)** is a process primarily used to adapt pre-trained language models to **follow instructions**, engage in dialogue, and use specific output formats.
- While pre-trained models have impressive general capabilities, `SFT helps transform them into assistant-like models that can better understand and respond to user prompts`. This is typically done by training on datasets of human-written conversations and instructions.
- SFT involves significant computational resources and engineering effort, so it should only be pursued when prompting existing models proves insufficient.

If you determine that SFT is necessary, the decision to proceed depends on two primary factors:

- Template Control

SFT allows precise control over the model’s output structure. This is particularly valuable when you need the model to:

1. Generate responses in a specific chat template format
2. Follow strict output schemas
3. Maintain consistent styling across responses

- Domain Adaptation

When working in specialized domains, SFT helps align the model with domain-specific requirements by:

1. Teaching domain terminology and concepts
2. Enforcing professional standards
3. Handling technical queries appropriately
4. Following industry-specific guidelines


- Dataset Preparation

The supervised fine-tuning process requires a task-specific dataset structured with input-output pairs. Each pair should consist of:

1. An input prompt
2. The expected model response
3. Any additional context or metadata

The quality of your training data is crucial for successful fine-tuning. Let’s look at how to prepare and validate your dataset:

## Training Configuration

The success of your fine-tuning depends heavily on choosing the right training parameters. Let’s explore each important parameter and how to configure them effectively:

The SFTTrainer configuration requires consideration of several parameters that control the training process. Let’s explore each parameter and their purpose:

Training Duration Parameters:

`num_train_epochs`: Controls total training duration

`max_steps`: Alternative to epochs, sets maximum number of training steps
More epochs allow better learning but risk overfitting
Batch Size Parameters:

`per_device_train_batch_size`: Determines memory usage and training stability

`gradient_accumulation_steps`: Enables larger effective batch sizes
Larger batches provide more stable gradients but require more memory
Learning Rate Parameters:

`learning_rate`: Controls size of weight updates

`warmup_ratio`: Portion of training used for learning rate warmup
Too high can cause instability, too low results in slow learning
Monitoring Parameters:

`logging_steps`: Frequency of metric logging

`eval_steps`: How often to evaluate on validation data

`save_steps`: Frequency of model checkpoint saves

Start with conservative values and adjust based on monitoring: - Begin with 1-3 epochs - Use smaller batch sizes initially - Monitor validation metrics closely - Adjust learning rate if training is unstable

# Transformers Reinforcement Learning (TRL) library

In [ ]:
from datasets import load_dataset
from trl import SFTConfig,SFTTrainer
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
dataset = load_dataset("HuggingFaceTB/smoltalk","all")
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)


In [ ]:
from trl import setup_chat_format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [ ]:
training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer, #handle tokenization automatically
)


When using a dataset with a "messages" field (like the example above), the SFTTrainer automatically applies the model's chat template, which it retrieves from the hub. This means you don't need any additional configuration to handle chat-style conversations - the trainer will format the messages according to the model's expected template format.

In [ ]:
trainer.train()

| Feature/Use Case                | `transformers.Trainer`                | `trl.SFTTrainer`                     |
| ------------------------------- | ------------------------------------- | ------------------------------------ |
| Model Type                      | Any (BERT, T5, GPT, etc.)             | Primarily causal LMs (GPT-like chat) |
| Dataset Format                  | Tokenized inputs (input\_ids, labels) | Raw chat messages (`messages` field) |
| Tokenization                    | Manual or external                    | Automatic via `processing_class`     |
| Chat/Instruction Formatting     | None                                  | Built-in with chat templates         |
| Reinforcement Learning Support  | No                                    | Yes (PPO, DPO, SFT)                  |
| Ease for Chat Model Fine-tuning | Requires manual setup                 | Out-of-the-box support               |


| Argument                                 | What it controls                     | How to choose                                                                             |
| ---------------------------------------- | ------------------------------------ | ----------------------------------------------------------------------------------------- |
| **learning\_rate**                       | Step size for optimizer updates      | Start with typical defaults like 5e-5 or 3e-4; lower for bigger models or sensitive tasks |
| **per\_device\_train\_batch\_size**      | Batch size per GPU/CPU device        | Depends on GPU memory; bigger batch sizes help convergence but require more memory        |
| **max\_steps** or **num\_train\_epochs** | Total training duration              | `max_steps` for fixed steps, `num_train_epochs` for full passes over dataset              |
| **warmup\_steps**                        | Steps to gradually increase LR       | Usually 5-10% of total steps; helps stable training start                                 |
| **logging\_steps**                       | How often to log metrics             | Every 10-100 steps, depending on training speed                                           |
| **eval\_steps**                          | How often to evaluate model          | Balance between feedback and overhead; e.g., every 50-100 steps                           |
| **save\_steps**                          | How often to save checkpoints        | Similar to eval steps or longer if saving is slow                                         |
| **weight\_decay**                        | Regularization to reduce overfitting | Typical values 0.01 or 0; adjust if overfitting                                           |


| Feature               | Encoder-Only  | Decoder-Only   | Encoder-Decoder                            |
| --------------------- | ------------- | -------------- | ------------------------------------------ |
| Direction             | Bidirectional | Unidirectional | Bidirectional (enc) → Unidirectional (dec) |
| Pretraining Objective | Masked LM     | Causal LM      | Denoising / Seq2Seq                        |
| Text Generation       | ❌ Not suited  | ✅ Excellent    | ✅ Excellent                                |
| Classification Tasks  | ✅ Excellent   | ❌ Not ideal    | ✅ With prompt tuning                       |
| Output Control        | No generation | Token by token | Token by token                             |
